In [1]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    BertTokenizerFast,
    BertForSequenceClassification,
    BertTokenizer,
    AutoTokenizer
)
from huggingface_hub import HfFolder, notebook_login
import os

os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

2024-04-21 04:45:58.352266: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 04:45:58.357848: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 04:45:58.418264: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 04:46:00.104919: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from datasets import Dataset
import pandas as pd

full_dataset = pd.read_csv("round5.csv", nrows=20000)

In [3]:
# model_id = "roberta-base"
model_id = "bert-base-uncased"

In [4]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_dataset, test_dataset = train_test_split(full_dataset, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

In [5]:
dataset = Dataset.from_pandas(full_dataset)

In [6]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Assuming 'predicted' is the column with class names
unique_classes = full_dataset['predicted'].unique()
class_to_index = {label: idx for idx, label in enumerate(unique_classes)}

# Add this to your tokenize function to use this mapping
def tokenize(batch):
    tokenized_inputs = tokenizer(batch['concatenated_text'], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs['labels'] = torch.tensor([class_to_index[label] for label in batch['labels']], dtype=torch.long)
    return tokenized_inputs



# Tokenize the datasets
train_dataset = train_dataset.rename_column('predicted', 'labels')
test_dataset = test_dataset.rename_column('predicted', 'labels')
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# You might need to set format for PyTorch

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [7]:
full_dataset

,concatenated_text,predicted
0,White Horse;Tile Manufacturing | European Aest...,Building Material and Garden Equipment and Sup...
1,Wealth Solution Partners;Super and SMSF Servic...,"Funds, Trusts, and Other Financial Vehicles"
2,PMG;Fire and Water Cleanup Services | Mold Rem...,Waste Management and Remediation Services
3,TMP Capital PLLC;Licensed in AL & FL | 203K Lo...,Credit Intermediation and Related Activities
4,Genertek Power;Industrial and Commercial Energ...,Utilities
...,...,...
19995,Pros DIY;Kitchen Sink Drains | DIY Project Tut...,Repair and Maintenance
19996,Dajo Hair Salon Studio;Personal Care Services ...,Personal and Laundry Services
19997,Big Accessories;Customization Services for Bag...,"Sporting Goods, Hobby, Musical Instrument, Boo..."
19998,Canadian Urban;Real Estate Syndication | Real ...,Real Estate


In [8]:
repository_id = "output"

!nvidia-smi

# Model
model = BertForSequenceClassification.from_pretrained(model_id)

model = model.to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun Apr 21 04:46:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:1A:00.0 Off |                  Off |
|  0%   29C    P8    10W / 450W |      3MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:3D:00.0 Off |                  Off |
|  0%   

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from datasets import load_metric
import numpy as np
from transformers import Trainer, TrainingArguments

# data collator is used to collate the data into batches that can be fed to the model during training and evaluation
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

# define metrics and metrics function
f1_metric = load_metric("f1")
accuracy_metric = load_metric( "accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="micro")
    return {
        "accuracy": acc["accuracy"],
        "f1": f1["f1"],
    }

training_args = TrainingArguments(
    output_dir='output_dir',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,  # Make sure this function is appropriate for your classification
    tokenizer = tokenizer,
    data_collator=data_collator
)

trainer.train()

/root/tmp/ipykernel_6745/1189197698.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1_metric = load_metric("f1")
/root/.conda/envs/veridion/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/root/.conda/envs/veridion/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You

TypeError: len() of a 0-d tensor

In [ ]:
for batch in train_dataset:
    print(batch['input_ids'].shape, batch['attention_mask'].shape, batch['labels'])
    break  # Just print the first batch to check

In [ ]:
trainer.train()